In [3]:
import flair.datasets
from pathlib import Path
from flair.data import Corpus
from flair.datasets import ColumnCorpus
from flair.data_fetcher import NLPTaskDataFetcher, NLPTask
from flair.embeddings import TokenEmbeddings, BertEmbeddings,  ELMoEmbeddings, FlairEmbeddings, WordEmbeddings, PooledFlairEmbeddings, StackedEmbeddings, CharacterEmbeddings, CharLMEmbeddings
from flair.models import LanguageModel
from gensim.models.keyedvectors import KeyedVectors
from typing import List
import os

In [4]:
# Disease
# NCBI_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/NCBI-disease-IOB'
#BioNLP13PC_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BioNLP13PC-IOB'
# BC5CDR_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC5CDR-disease-IOB'
# Genes
# BC2GM_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/BC2GM-IOB'
# JNLPBA_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/JNLPBA'

# Organisms
linnaeus_data_folder = '/nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/linnaeus-IOB'

# define columns
columns = {0: 'text', 1: 'ner'}

In [5]:
# NCBI = ColumnCorpus(NCBI_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

# BioNLP13PC = ColumnCorpus(BioNLP13PC_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

# BC5CDR = ColumnCorpus(BC5CDR_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)


# BC2GM = ColumnCorpus(BC2GM_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)


# JNLPBA = ColumnCorpus(JNLPBA_data_folder, columns, 
#                               train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

linnaeus = ColumnCorpus(linnaeus_data_folder, columns, 
                              train_file='train.tsv',  test_file='test.tsv', dev_file='devel.tsv', in_memory=False)

2019-10-07 15:56:06,826 Reading data from /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/linnaeus-IOB
2019-10-07 15:56:06,829 Train: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/linnaeus-IOB/train.tsv
2019-10-07 15:56:06,830 Dev: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/linnaeus-IOB/devel.tsv
2019-10-07 15:56:06,832 Test: /nfs/gns/literature/Santosh_Tirunagari/NER_Datasets/linnaeus-IOB/test.tsv


In [6]:
# from flair.data import MultiCorpus
# multi_corpus = MultiCorpus([NCBI, BC5CDR])
# multi_corpus = MultiCorpus([BC2GM])

In [7]:
#corpus: TaggedCorpus = read_data()
# print(multi_corpus)

In [8]:
tag_type = 'ner'

# 3. make the tag dictionary from the corpus
tag_dictionary = linnaeus.make_tag_dictionary(tag_type=tag_type)
print(tag_dictionary.idx2item)

[b'<unk>', b'O', b'B-Species', b'I-Species', b'<START>', b'<STOP>']


In [9]:
print(linnaeus)

Corpus: 11935 train + 4078 dev + 7142 test sentences


In [10]:
# embedding_types: List[TokenEmbeddings] = [

#     WordEmbeddings('glove'),

# #     pretrained_word2vec,

#     # comment in this line to use character embeddings
#     # CharacterEmbeddings(),

#     # comment in these lines to use contextual string embeddings
#     FlairEmbeddings('news-forward'),
#     FlairEmbeddings('news-backward'),
# ]

# word_embeddings = [WordEmbeddings('glove'), FlairEmbeddings('pubmed-forward'), 
# FlairEmbeddings('pubmed-backward'), FlairEmbeddings('news-forward-fast'), FlairEmbeddings('news-backward-fast')]

elmo_embeddings_pubmed = ELMoEmbeddings('pubmed')

embedding_types: List[TokenEmbeddings] = [
                   WordEmbeddings('glove'),
                   FlairEmbeddings('news-forward'),
                   FlairEmbeddings('news-backward'),
                   CharacterEmbeddings(), 
                   PooledFlairEmbeddings('pubmed-backward', pooling='max'),
                   PooledFlairEmbeddings('pubmed-forward', pooling='max'),#
                   ]

In [11]:
embeddings: StackedEmbeddings = StackedEmbeddings(embeddings=embedding_types)

In [12]:
# 5. initialize sequence tagger
from flair.models import SequenceTagger

tagger: SequenceTagger = SequenceTagger(hidden_size=256,
                                        embeddings=embeddings,
                                        tag_dictionary=tag_dictionary,
                                        tag_type=tag_type,
                                        use_crf=True)

In [14]:
# 6. initialize trainer
from flair.trainers import ModelTrainer

trainer: ModelTrainer = ModelTrainer(tagger, linnaeus)

In [ ]:
# 7. start training
# data_path = '/nfs/gns/literature/Santosh_Tirunagari/'
# if os.path.isdir(data_path+'/Models/') == False:
#     os.mkdir(data_path+'/Models/ner_flair_model')

In [15]:
trainer.train('../flair_models/ner/multi_bio_ner_model_organisms/v01',
              learning_rate=0.1,
              mini_batch_size=32,
              max_epochs=150)

2019-10-07 16:02:09,910 ----------------------------------------------------------------------------------------------------
2019-10-07 16:02:09,912 Model: "SequenceTagger(
  (embeddings): StackedEmbeddings(
    (list_embedding_0): WordEmbeddings('glove')
    (list_embedding_1): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_2): FlairEmbeddings(
      (lm): LanguageModel(
        (drop): Dropout(p=0.05)
        (encoder): Embedding(300, 100)
        (rnn): LSTM(100, 2048)
        (decoder): Linear(in_features=2048, out_features=300, bias=True)
      )
    )
    (list_embedding_3): CharacterEmbeddings(
      (char_embedding): Embedding(275, 25)
      (char_rnn): LSTM(25, 25, bidirectional=True)
    )
    (list_embedding_4): PooledFlairEmbeddings(
      (context_embeddings): FlairEmbed

2019-10-07 16:36:11,222 epoch 4 - iter 222/373 - loss 0.10851340 - samples/sec: 26.60
2019-10-07 16:36:56,454 epoch 4 - iter 259/373 - loss 0.10631703 - samples/sec: 27.04
2019-10-07 16:37:42,274 epoch 4 - iter 296/373 - loss 0.10342955 - samples/sec: 27.12
2019-10-07 16:38:26,172 epoch 4 - iter 333/373 - loss 0.10544793 - samples/sec: 28.19
2019-10-07 16:39:09,655 epoch 4 - iter 370/373 - loss 0.10568820 - samples/sec: 28.45
2019-10-07 16:39:13,759 ----------------------------------------------------------------------------------------------------
2019-10-07 16:39:13,761 EPOCH 4 done: loss 0.1054 - lr 0.1000
2019-10-07 16:41:02,510 DEV : loss 0.08282675594091415 - score 0.92
2019-10-07 16:41:12,387 BAD EPOCHS (no improvement): 1
2019-10-07 16:41:12,400 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-10-07 16:41:15,393 epoch 5 - iter 0/373 - loss 0.01436006 - sample

2019-10-07 17:36:38,696 epoch 10 - iter 37/373 - loss 0.07221936 - samples/sec: 25.74
2019-10-07 17:37:24,254 epoch 10 - iter 74/373 - loss 0.06733975 - samples/sec: 26.88
2019-10-07 17:38:08,199 epoch 10 - iter 111/373 - loss 0.06185794 - samples/sec: 28.58
2019-10-07 17:38:50,799 epoch 10 - iter 148/373 - loss 0.05949125 - samples/sec: 29.00
2019-10-07 17:39:33,571 epoch 10 - iter 185/373 - loss 0.06134819 - samples/sec: 28.71
2019-10-07 17:40:17,175 epoch 10 - iter 222/373 - loss 0.06347091 - samples/sec: 28.69
2019-10-07 17:41:01,883 epoch 10 - iter 259/373 - loss 0.06194271 - samples/sec: 28.32
2019-10-07 17:41:47,968 epoch 10 - iter 296/373 - loss 0.06354924 - samples/sec: 26.50
2019-10-07 17:42:34,116 epoch 10 - iter 333/373 - loss 0.06592136 - samples/sec: 26.42
2019-10-07 17:43:19,601 epoch 10 - iter 370/373 - loss 0.06604388 - samples/sec: 27.21
2019-10-07 17:43:23,404 ----------------------------------------------------------------------------------------------------
2019-10

2019-10-07 18:31:36,489 ----------------------------------------------------------------------------------------------------
2019-10-07 18:31:36,491 EPOCH 15 done: loss 0.0474 - lr 0.0500
2019-10-07 18:33:30,726 DEV : loss 0.05392361432313919 - score 0.9652
2019-10-07 18:33:38,723 BAD EPOCHS (no improvement): 3
2019-10-07 18:33:38,732 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-10-07 18:33:41,805 epoch 16 - iter 0/373 - loss 0.12944353 - samples/sec: 1117.52
2019-10-07 18:34:27,101 epoch 16 - iter 37/373 - loss 0.05685005 - samples/sec: 26.99
2019-10-07 18:35:08,810 epoch 16 - iter 74/373 - loss 0.05714187 - samples/sec: 29.36
2019-10-07 18:35:52,123 epoch 16 - iter 111/373 - loss 0.05645856 - samples/sec: 28.77
2019-10-07 18:36:36,588 epoch 16 - iter 148/373 - loss 0.06249594 - samples/sec: 28.17
2019-10-07 18:37:23,794 epoch 16 - iter 185/373 - loss 0.06274816

2019-10-07 19:24:33,702 epoch 21 - iter 148/373 - loss 0.04767703 - samples/sec: 26.59
2019-10-07 19:25:17,368 epoch 21 - iter 185/373 - loss 0.04727083 - samples/sec: 28.58
2019-10-07 19:26:04,072 epoch 21 - iter 222/373 - loss 0.04803490 - samples/sec: 26.88
2019-10-07 19:26:51,177 epoch 21 - iter 259/373 - loss 0.04638595 - samples/sec: 25.90
2019-10-07 19:27:36,853 epoch 21 - iter 296/373 - loss 0.04756179 - samples/sec: 27.46
2019-10-07 19:28:23,751 epoch 21 - iter 333/373 - loss 0.04686910 - samples/sec: 26.29
2019-10-07 19:29:08,370 epoch 21 - iter 370/373 - loss 0.04679646 - samples/sec: 27.40
2019-10-07 19:29:12,042 ----------------------------------------------------------------------------------------------------
2019-10-07 19:29:12,044 EPOCH 21 done: loss 0.0468 - lr 0.0125
2019-10-07 19:30:59,866 DEV : loss 0.05417634919285774 - score 0.9672
2019-10-07 19:31:07,077 BAD EPOCHS (no improvement): 1
2019-10-07 19:31:07,084 ------------------------------------------------------

...

2019-10-07 21:11:04,388 epoch 32 - iter 259/373 - loss 0.03159720 - samples/sec: 27.52
2019-10-07 21:11:47,552 epoch 32 - iter 296/373 - loss 0.03276620 - samples/sec: 28.16
2019-10-07 21:12:32,983 epoch 32 - iter 333/373 - loss 0.03489102 - samples/sec: 27.14
2019-10-07 21:13:19,480 epoch 32 - iter 370/373 - loss 0.03641315 - samples/sec: 26.13
2019-10-07 21:13:23,215 ----------------------------------------------------------------------------------------------------
2019-10-07 21:13:23,218 EPOCH 32 done: loss 0.0362 - lr 0.0031
2019-10-07 21:15:07,901 DEV : loss 0.053491126745939255 - score 0.9687
Epoch    31: reducing learning rate of group 0 to 1.5625e-03.
2019-10-07 21:15:15,248 BAD EPOCHS (no improvement): 4
2019-10-07 21:15:15,253 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-10-07 21:15:18,826 epoch 33 - iter 0/373 - loss 0.19968280 - samples/sec: 751.57
2

train mode resetting embeddings
train mode resetting embeddings
2019-10-07 22:02:45,519 epoch 38 - iter 0/373 - loss 0.00143003 - samples/sec: 1123.88
2019-10-07 22:03:32,689 epoch 38 - iter 37/373 - loss 0.08248596 - samples/sec: 26.06
2019-10-07 22:04:21,098 epoch 38 - iter 74/373 - loss 0.05589074 - samples/sec: 25.64
2019-10-07 22:05:08,640 epoch 38 - iter 111/373 - loss 0.04863003 - samples/sec: 26.45
2019-10-07 22:05:53,153 epoch 38 - iter 148/373 - loss 0.04762088 - samples/sec: 27.87
2019-10-07 22:06:39,547 epoch 38 - iter 185/373 - loss 0.04661623 - samples/sec: 26.46
2019-10-07 22:07:24,690 epoch 38 - iter 222/373 - loss 0.04324069 - samples/sec: 27.08
2019-10-07 22:08:08,991 epoch 38 - iter 259/373 - loss 0.04325279 - samples/sec: 27.95
2019-10-07 22:08:53,385 epoch 38 - iter 296/373 - loss 0.04136435 - samples/sec: 27.85
2019-10-07 22:09:38,469 epoch 38 - iter 333/373 - loss 0.04175414 - samples/sec: 27.19
2019-10-07 22:10:22,728 epoch 38 - iter 370/373 - loss 0.04088574 - 

2019-10-07 22:57:05,502 epoch 43 - iter 333/373 - loss 0.03626819 - samples/sec: 29.08
2019-10-07 22:57:48,266 epoch 43 - iter 370/373 - loss 0.03493779 - samples/sec: 28.86
2019-10-07 22:57:52,296 ----------------------------------------------------------------------------------------------------
2019-10-07 22:57:52,298 EPOCH 43 done: loss 0.0349 - lr 0.0004
2019-10-07 22:59:36,739 DEV : loss 0.055269040167331696 - score 0.9672
2019-10-07 22:59:44,418 BAD EPOCHS (no improvement): 3
2019-10-07 22:59:44,428 ----------------------------------------------------------------------------------------------------
train mode resetting embeddings
train mode resetting embeddings
2019-10-07 22:59:47,012 epoch 44 - iter 0/373 - loss 0.01979747 - samples/sec: 1110.84
2019-10-07 23:00:31,351 epoch 44 - iter 37/373 - loss 0.03597614 - samples/sec: 28.09
2019-10-07 23:01:17,487 epoch 44 - iter 74/373 - loss 0.04302022 - samples/sec: 26.98
2019-10-07 23:01:59,801 epoch 44 - iter 111/373 - loss 0.0400848

2019-10-07 23:47:59,231 ----------------------------------------------------------------------------------------------------
2019-10-07 23:48:08,939 ----------------------------------------------------------------------------------------------------
2019-10-07 23:48:08,941 Testing using best model ...
2019-10-07 23:48:08,945 loading file ../flair_models/ner/multi_bio_ner_model_organisms/v01/best-model.pt
2019-10-07 23:51:37,035 0.8921	0.8423	0.8665
2019-10-07 23:51:37,038 
MICRO_AVG: acc 0.7644 - f1-score 0.8665
MACRO_AVG: acc 0.7644 - f1-score 0.8665
Species    tp: 1207 - fp: 146 - fn: 226 - tn: 1207 - precision: 0.8921 - recall: 0.8423 - accuracy: 0.7644 - f1-score: 0.8665
2019-10-07 23:51:37,040 ----------------------------------------------------------------------------------------------------


{'test_score': 0.8665,
 'dev_score_history': [0.8569,
  0.8726,
  0.9604,
  0.92,
  0.9615,
  0.9515,
  0.9437,
  0.9739,
  0.9678,
  0.9708,
  0.9673,
  0.9674,
  0.9638,
  0.9659,
  0.9652,
  0.965,
  0.9687,
  0.9665,
  0.968,
  0.968,
  0.9672,
  0.9673,
  0.9666,
  0.9679,
  0.9665,
  0.968,
  0.9672,
  0.9665,
  0.9665,
  0.9665,
  0.9665,
  0.9687,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9679,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9672,
  0.9672],
 'train_loss_history': [0.522413571582403,
  0.1860033163236549,
  0.14110816024423603,
  0.10538855398288521,
  0.1007870908516981,
  0.09396663953327941,
  0.08209875103176119,
  0.07643543006028311,
  0.07370142554789381,
  0.06653031497075156,
  0.05859969911944551,
  0.06154421062755521,
  0.05407748500449488,
  0.04966888293354265,
  0.04737863979045172,
  0.05741916834089615,
  0.04682797660936299,
  0.04178646923075293,
  0.04240656164212617,
  0.041491293875205934,
  

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence, Token
from flair.models import SequenceTagger
# load the model you trained
# gene_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_gene/best-model.pt')
# disease_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_disease/best-model.pt')


# create example sentence
sentence= 'Conditions for production, and some characteristics, of mycobacterial growth inhibitory factor produced by spleen cells from mice immunized with viable cells of the attenuated H37Ra strain of Mycobacterium tuberculosis.'

gene_sentence = Sentence(sentence)
gene_model.predict(gene_sentence)

disease_sentence = Sentence(sentence)
disease_model.predict(disease_sentence)

In [ ]:
str(gene_sentence.get_spans('ner')).split(':')[1].replace('"','').replace(']','').replace('>','').strip()

In [ ]:
str(disease_sentence.get_spans('ner')).split(':')[1].replace('"','').replace(']','').replace('>','').strip()

In [ ]:
from flair.models import TextClassifier
from flair.data import Sentence, Token

# use a library to split into sentences
from segtok.segmenter import split_single
sentences = [Sentence(sent, use_tokenizer=True) for sent in split_single(text)]



# load the model you trained
# disease_model = SequenceTagger.load('../flair_models/ner/multi_bio_ner_model_disease/best-model.pt')

# create example sentence
xx= 'For example, the chemical inhibition of IL4I1 activity may represent a new adjuvant strategy for the treatment of cancer by restoring specific anti-tumor immune responses.'

xx1 = 'These results suggested that during ischemic cardiomyopathy, initially neutrophil proteinase activates latent myocardial MMP which can degrade ECM, which continuously degrades if not controlled by TIMP, leading to ventricular dilatation and dysfunction.'

xx2= 'GR3027 antagonizes GABAA receptor potentiating neurosteroids and restores spatial learning and motor coordination in rats with chronic hyperammonemia and hepatic encephalopathy'

sentence = Sentence(xx2)

# predict tags and print
disease_model.predict(sentence)

print(sentence.to_tagged_string())